# Power & Sample Size Playground (Widgets)

Interact with parameters to see required N, MOE-driven N, false-positive compounding, and simulation-based power.

In [ ]:
!pip -q install statsmodels ipywidgets numpy scipy matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from ipywidgets import interact, FloatSlider, IntSlider
from statsmodels.stats.power import TTestIndPower
from math import ceil


In [ ]:
def ttest_power_widget(effect_raw=0.5, sd=1.2, alpha=0.05, target_power=0.8):
    d = effect_raw / sd
    analysis = TTestIndPower()
    n_per_group = analysis.solve_power(effect_size=d, alpha=alpha, power=target_power, alternative='two-sided')
    print(f"Cohen's d = {d:.3f}")
    print(f"Required N per group ≈ {ceil(n_per_group)} (total ≈ {2*ceil(n_per_group)})")

interact(
    ttest_power_widget,
    effect_raw=FloatSlider(description="Δ (mean diff)", min=0.05, max=1.0, step=0.05, value=0.5),
    sd=FloatSlider(description="σ (SD)", min=0.4, max=2.0, step=0.05, value=1.2),
    alpha=FloatSlider(description="α", min=0.001, max=0.10, step=0.001, value=0.05, readout_format=".3f"),
    target_power=FloatSlider(description="Power", min=0.5, max=0.99, step=0.01, value=0.80)
);


In [ ]:
def proportion_moe_widget(p=0.5, moe=0.05, alpha=0.05, finite_pop=0):
    z = stats.norm.ppf(1 - alpha/2)
    var = p*(1-p)
    n = (z**2 * var) / (moe**2)
    if finite_pop and finite_pop > 0:
        n = n / (1 + (n - 1)/finite_pop)
    print(f"Required n ≈ {ceil(n)}")
    if p == 0.5:
        print("(Worst-case variance; if you have a better p estimate, n may be smaller.)")

interact(
    proportion_moe_widget,
    p=FloatSlider(description="p (expected)", min=0.05, max=0.95, step=0.01, value=0.50),
    moe=FloatSlider(description="MOE (±)", min=0.005, max=0.15, step=0.005, value=0.05, readout_format=".3f"),
    alpha=FloatSlider(description="α", min=0.001, max=0.10, step=0.001, value=0.05, readout_format=".3f"),
    finite_pop=IntSlider(description="Finite N (0=∞)", min=0, max=200000, step=1000, value=0)
);


In [ ]:
def fp_curve(alpha=0.05, max_tests=50):
    m = np.arange(1, max_tests+1)
    prob = 1 - (1 - alpha)**m
    plt.figure(figsize=(6,4))
    plt.plot(m, prob)
    plt.xlabel("# tests (m)")
    plt.ylabel("P(≥1 false positive)")
    plt.title(f"Familywise error vs. number of tests (α={alpha:.3f})")
    plt.grid(True)
    plt.show()

interact(
    fp_curve,
    alpha=FloatSlider(description="α", min=0.001, max=0.10, step=0.001, value=0.05, readout_format=".3f"),
    max_tests=IntSlider(description="max m", min=5, max=200, step=5, value=50)
);


In [ ]:
def simulate_power_widget(mu1=8.0, mu2=10.0, sd=6.0, n=200, reps=3000, alpha=0.05):
    import numpy as np
    from scipy import stats
    rng = np.random.default_rng(42)
    sd = float(sd)
    k = (mu1**2) / max(sd**2 - mu1, 1e-6)
    successes = 0
    for _ in range(int(reps)):
        rate_a = rng.gamma(shape=max(k,1e-3), scale=mu1/max(k,1e-3), size=n)
        rate_b = rng.gamma(shape=max(k,1e-3), scale=mu2/max(k,1e-3), size=n)
        a = rng.poisson(rate_a)
        b = rng.poisson(rate_b)
        p = stats.ttest_ind(a, b, equal_var=False).pvalue
        successes += (p < alpha)
    power = successes / reps
    print(f"Estimated power ≈ {power:.2f} (reps={reps}, N per group={n})")

interact(
    simulate_power_widget,
    mu1=FloatSlider(description="μA", min=1.0, max=20.0, step=0.5, value=8.0),
    mu2=FloatSlider(description="μB", min=1.0, max=20.0, step=0.5, value=10.0),
    sd=FloatSlider(description="SD approx", min=1.0, max=12.0, step=0.5, value=6.0),
    n=IntSlider(description="N/group", min=20, max=1000, step=20, value=200),
    reps=IntSlider(description="reps", min=500, max=10000, step=500, value=3000),
    alpha=FloatSlider(description="α", min=0.001, max=0.10, step=0.001, value=0.05, readout_format=".3f")
);
